In [ ]:
# Simple neural network, that uses convolutions and downsampling to genneralize

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torchvision import datasets, transforms

In [ ]:
data_path = r"/content/drive/My Drive/cifar10/data"
data_path = data_path.replace('\\', '/')

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
tensor_cifar10 = datasets.CIFAR10(data_path, 
                                  train = True, 
                                  download = True, 
                                  transform = transforms.ToTensor())


tensor_cifar10

Extracting /content/drive/My Drive/cifar10/data/cifar-10-python.tar.gz to /content/drive/My Drive/cifar10/data


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /content/drive/My Drive/cifar10/data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
imgs = torch.stack([img_t for img_t, label in tensor_cifar10], dim=3)
#torch.stack(tensors, dim=0, *, out=None) → Tensor
#Concatenates sequence of tensors along a new dimension.
#All tensors need to be of the same size.

In [ ]:
imgs.shape #rgb, h, w, datapoints

torch.Size([3, 32, 32, 50000])

In [ ]:
mean = imgs.view(3, -1).mean(dim=1)
mean
#Recall that view(3, -1) keeps the three channels and
#merges all the remaining dimensions into one, figuring
#out the appropriate size. Here our 3 × 32 × 32 x 50000 image is
#transformed into a 3 × 1,024 vector, and then the mean
#is taken over the 32 × 32 x 50000 elements of each channel

tensor([0.4914, 0.4822, 0.4465])

In [ ]:
std = imgs.view(3, -1).std(dim=1) # dim is which dimension index to do the calc on
std

tensor([0.2470, 0.2435, 0.2616])

In [ ]:
from torchvision import datasets, transforms
data_path = data_path
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]))

Files already downloaded and verified


In [ ]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))

Files already downloaded and verified


In [ ]:
class_names = ['airplane', 'bird']
label_map = {0: 0, 2: 1}

cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [ ]:
#convolutionary feed forward network that uses dropout to combat overfitting
import torch.nn.functional as fn

class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.4) # probability for dropout 40%
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.4) # probability for dropout 40%
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2) # convolution, tanh activation, downsample by half
        out = self.conv1_dropout(out) # apply
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out) # apply
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, scheduler, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  #  starting from  1 instead of 0
        loss_train = 0.0
        for imgs, labels in train_loader:  # Using the train loader

            imgs = imgs.to(device)
            labels = labels.to(device)
            
            outputs = model(imgs)  # applying the model on the img data in train loader
            
            loss = loss_fn(outputs, labels)  # computing the loss

            optimizer.zero_grad()  # clearing the gradients from last pass
            
            loss.backward()  # compute gradients on the parameters
            
            optimizer.step()  # optimize model based on gradients

            loss_train += loss.item()  # sums losses over epoch, item() is to escape gradients

        scheduler.step()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # Divides by the length of the
                                                  #training data loader to get the
                                                #average loss per batch.

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(f"Training on device {device}.")
use_cuda = torch.cuda.is_available()

Training on device cuda.


In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, 
                                           batch_size=64, 
                                           pin_memory=use_cuda, #Pinned memory transfers to GPU quickly
                                           num_workers = 2, # collab cpu has 2 threads..
                                           shuffle=True)  # The DataLoader batches up the examples of our cifar2 dataset.
                                           #Shuffling randomizes the order of the examples from the dataset.

model = NetDropout()  #  instantiate network
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.82)  #  instantiate optimizer with model parameters and learning rate
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, gamma=0.8, step_size=20)# Decay LR by a factor of gamma every step_size
#Decays the learning rate of each parameter group by gamma every step_size epochs. Notice that such decay can happen. When last_epoch=-1, sets initial lr as lr.
loss_fn = nn.CrossEntropyLoss()  #  loss function

training_loop(  # call training loop
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    scheduler = exp_lr_scheduler,
    loss_fn = loss_fn,
    train_loader = train_loader
)

2020-11-21 22:11:22.913409 Epoch 1, Training loss 0.5112302293823023
2020-11-21 22:11:30.044592 Epoch 10, Training loss 0.32122074589607824
2020-11-21 22:11:38.131233 Epoch 20, Training loss 0.27138395355955053
2020-11-21 22:11:46.039676 Epoch 30, Training loss 0.23438411618873572
2020-11-21 22:11:54.025223 Epoch 40, Training loss 0.21414515015425956
2020-11-21 22:12:02.016983 Epoch 50, Training loss 0.1921895585337262
2020-11-21 22:12:10.286756 Epoch 60, Training loss 0.17706134568923598
2020-11-21 22:12:18.197728 Epoch 70, Training loss 0.16045254228079014
2020-11-21 22:12:25.887738 Epoch 80, Training loss 0.1508544286724868
2020-11-21 22:12:34.113153 Epoch 90, Training loss 0.13790187232528522
2020-11-21 22:12:42.147765 Epoch 100, Training loss 0.12926792127975992


In [ ]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, pin_memory=use_cuda, num_workers = 2, shuffle=False)

In [ ]:

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # we do not want to update parameters
            for imgs, labels in loader:
                imgs = imgs.to(device)
                labels = labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # give us the index of the highest value as output
                total += labels.shape[0]  # counts number of examples, so total is increased by batch size
                correct += int((predicted == labels).sum())  # Comparing the predicted class that had the
                                                            #maximum probability and the ground-truth
                                                            #labels, we first get a Boolean array. Taking the
                                                            #sum gives the number of items in the batch
                                                            #where the prediction and ground truth agree.

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.95
Accuracy val: 0.89
